In [39]:
import os

In [40]:
%pwd

'd:\\Mlops\\ML_Project'

In [41]:
%pwd

'd:\\Mlops\\ML_Project'

In [42]:
#Entity 

In [46]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir: Path

In [44]:
#Configuration Manager Code

In [48]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [57]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params_ = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config



In [ ]:
#Components Code

In [59]:
import os
import urllib.request as request
import zipfile
from mlProject.logging import logger
from mlProject.utils.common import get_size
from mlProject.config.configuration import *

In [69]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename = headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
#Pipeline code

In [73]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-04 23:17:09,390: INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-01-04 23:17:09,392: INFO : common: yaml file: params.yaml loaded successfully]
[2024-01-04 23:17:09,395: INFO : common: yaml file: schema.yaml loaded successfully]
[2024-01-04 23:17:09,397: INFO : common: created directory at: artifacts]
[2024-01-04 23:17:09,398: INFO : common: created directory at: artifacts/data_ingestion]
[2024-01-04 23:17:10,747: INFO : 524352395: ('artifacts/data_ingestion/data.zip', <http.client.HTTPMessage object at 0x0000019FC4D61D60>) download! with following info: 
 ('artifacts/data_ingestion/data.zip', <http.client.HTTPMessage object at 0x0000019FC4D61D60>)]
